In [91]:
from lxml import etree
import time
import requests
import random
import pandas as pd
import pypinyin
from tqdm import tqdm

# ! pip install pypinyin -i http://pypi.douban.com/simple --trusted-host pypi.douban.com
# ! pip install html5lib -i http://pypi.douban.com/simple --trusted-host pypi.douban.com

def delete_symbol(string):
    return string.replace("℃",'')


def chinese_to_piyin(chinese_string):
    return "".join([char[0] for char in pypinyin.pinyin(chinese_string, style=pypinyin.NORMAL)])

def weather_data_api(city,year,month):
    
    res = []
    headers={
        'cookie':'UM_distinctid=173e57a3e94139-0c7d141de955d6-15366656-13c680-173e57a3e956bd; CNZZDATA1275796416=834360230-1597278994-%7C1597278994; Hm_lvt_ab6a683aa97a52202eab5b3a9042a8d2=1597282861; cityPy=shanghai; cityPy_expire=1597887954; Hm_lpvt_ab6a683aa97a52202eab5b3a9042a8d2=1597283746'
     ,
        'referer': 'https://lishi.tianqi.com/fangshan/202001.html',
        'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36'
    }
    
    date=year+month
    pinyin_city = chinese_to_piyin(city)
    url='https://lishi.tianqi.com/{}/{}.html'.format(pinyin_city,date)
    print(url)
    respones= requests.request( method='get',url=url, headers=headers)
    content=respones.content.decode('utf-8')

    html = etree.HTML(content)
    detail=html.xpath('//ul[@class="thrui"]')
    
    for node in detail[0][:-1]:
        
        data_line = node.xpath('.//text()')
        parse_line = [element for element in data_line if "\r" not in element]
        date = parse_line[0].split()
        weekday = date[-1]
        yymmdd = date[0].split('-')
        year = yymmdd[0]
        month = yymmdd[1]
        day = yymmdd[-1]
        info = [delete_symbol(e) for e in parse_line[1:]]
        
        data_token = [year]+[month]+[day]+[weekday]+info+[city]
        #print(info)
        res.append(data_token)
    column_names=['年','月','日', '星期' ,'最高温度℃','最低温度℃','天气','风力','城市']
    return pd.DataFrame(columns=column_names,data=res)

def batch_get_weather_data(city_list=[],year_list=[],month_list=[],data_dir_path=''):
    city_bar = tqdm(city_list)
    year_bar = tqdm(year_list)
    month_bar = tqdm(month_list)
    for city in tqdm(city_list):
        city_bar.set_description("城市:\t%s"%city)
        for year in year_bar:
            year_bar.set_description("年份:\t%s"%year)
            for month in month_bar:
                month_bar.set_description("月份\t:%s"%year)
                #table = weather_data_api(city,year,month)
                saved_path =  "{}{}_{}{}{}".format(data_dir_path,chinese_to_piyin(city),year,month,'_weather_data.csv')
                #table.to_csv(saved_path)
                print(saved_path)
                
                

weather_data_api("合肥","2022","08")


# path = "/home/zhangmanman/Documents/weather_databases/"
# citys = ['合肥',"淮南"]
# years = ['2021']
# months = ['01','02','03','04','05','06','07','08','09','10','11','12']
# batch_get_weather_data(citys,years,months,path)

https://lishi.tianqi.com/hefei/202208.html


,年,月,日,星期,最高温度℃,最低温度℃,天气,风力,城市
0,2022,08,01,星期一,34,25,多云,东风 2级,合肥
1,2022,08,02,星期二,36,26,晴,东北风 1级,合肥
2,2022,08,03,星期三,38,28,晴,东风 1级,合肥
3,2022,08,04,星期四,37,28,晴,南风 2级,合肥
4,2022,08,05,星期五,37,29,晴,东南风 2级,合肥
5,2022,08,06,星期六,38,27,多云,南风 2级,合肥
6,2022,08,07,星期日,37,29,多云,南风 2级,合肥
7,2022,08,08,星期一,38,27,多云,南风 3级,合肥
8,2022,08,09,星期二,38,27,多云,南风 2级,合肥
9,2022,08,10,星期三,37,27,多云,西南风 2级,合肥


## 早期版本代码 

In [75]:
# headers={
#     'cookie':'UM_distinctid=173e57a3e94139-0c7d141de955d6-15366656-13c680-173e57a3e956bd; CNZZDATA1275796416=834360230-1597278994-%7C1597278994; Hm_lvt_ab6a683aa97a52202eab5b3a9042a8d2=1597282861; cityPy=shanghai; cityPy_expire=1597887954; Hm_lpvt_ab6a683aa97a52202eab5b3a9042a8d2=1597283746'
#  ,
#     'referer': 'https://lishi.tianqi.com/fangshan/202001.html',
#     'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36'
# }
# list_=[]
# for x in ['hefei']:
#     for i in ['2018','2019','2020','2021','2022']:
#         for j in ['01','02','03','04','05','06','07','08','09','10','11','12']:
#             time.sleep(random.random()*2)
#             date=i+j
#             url='https://lishi.tianqi.com/{}/{}.html'.format(x,date)
#             print(url)
#             respones= requests.request( method='get',url=url, headers=headers)
#             content=respones.content.decode('utf-8')
#             html = etree.HTML(content)
#             detail=html.xpath('//ul[@class="thrui"]')
#             for n in detail[0][:-1]:
#                 print(n.xpath('.//text()')[1::2])
#                 list_.append(n.xpath('.//text()')[1::2]+[x])
# df=pd.DataFrame(columns=['date','最高温度','最低温度','天气','风力','城市'],data=list_)

 https://www.tianqi.com/hefei/

In [82]:
from tqdm import tqdm
import time

for i in tqdm(range(10),dynamic_ncols=, disable=True):
    i

In [109]:
def test_get_city():
    url = "https://www.tianqi.com/chinacity.html"
    headers={
    'cookie':'UM_distinctid=173e57a3e94139-0c7d141de955d6-15366656-13c680-173e57a3e956bd; CNZZDATA1275796416=834360230-1597278994-%7C1597278994; Hm_lvt_ab6a683aa97a52202eab5b3a9042a8d2=1597282861; cityPy=shanghai; cityPy_expire=1597887954; Hm_lpvt_ab6a683aa97a52202eab5b3a9042a8d2=1597283746'
     ,
    'referer': 'https://lishi.tianqi.com/fangshan/202001.html',
    'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36'
     }
    respones= requests.request( method='get',url=url, headers=headers)
    content=respones.content.decode('utf-8')
    regex_ = lambda string : re.findall(r'/">(.*)</a><a href="',string)
    step1 = regex_(content)[0]
    return pd.DataFrame({"城市":[ e.split('">')[-1] for e in re.split('</a><a href=',step1)]})

test_get_city().to_csv("city_name_table.csv")
        

,城市
0,海淀
1,朝阳
2,顺义
3,怀柔
4,通州
...,...
2577,鹿城
2578,龙湾
2579,嵊泗
2580,岱山


In [112]:
city_table = pd.read_csv("/home/zhangmanman/Documents/weather_databases/city_name_table.csv").城市.to_list()

In [113]:
city_table

['海淀',
 '朝阳',
 '顺义',
 '怀柔',
 '通州',
 '昌平',
 '延庆',
 '丰台',
 '石景山',
 '大兴',
 '房山',
 '密云',
 '门头沟',
 '平谷',
 '东城',
 '江北',
 '渝中',
 '大渡口',
 '沙坪坝',
 '九龙坡',
 '南岸',
 '开州',
 '永川',
 '合川',
 '南川',
 '江津',
 '万盛',
 '渝北',
 '北碚',
 '巴南',
 '长寿',
 '黔江',
 '万州',
 '涪陵',
 '开县',
 '城口',
 '云阳',
 '巫溪',
 '奉节',
 '巫山',
 '潼南',
 '垫江',
 '梁平',
 '忠县',
 '石柱',
 '大足',
 '荣昌',
 '铜梁',
 '璧山',
 '丰都',
 '武隆',
 '彭水',
 '綦江',
 '酉阳',
 '徐汇',
 '静安',
 '杨浦',
 '闵行',
 '宝山',
 '嘉定',
 '南汇',
 '金山',
 '青浦',
 '松江',
 '奉贤',
 '崇明',
 '徐家汇',
 '浦东',
 '长宁',
 '虹口',
 '武清',
 '宝坻',
 '东丽',
 '西青',
 '北辰',
 '宁河',
 '汉沽',
 '静海',
 '津南',
 '塘沽',
 '大港',
 '蓟州',
 '红桥',
 '河西',
 '河东',
 '河北',
 '和平',
 '南开',
 '枞阳',
 '太湖',
 '潜山',
 '怀宁',
 '宿松',
 '望江',
 '岳西',
 '桐城',
 '迎江',
 '宜秀',
 '怀远',
 '固镇',
 '五河',
 '龙子湖',
 '禹会',
 '淮上',
 '涡阳',
 '利辛',
 '蒙城',
 '庐江',
 '无为',
 '含山',
 '东至',
 '青阳',
 '九华山',
 '石台',
 '凤阳',
 '明光',
 '定远',
 '全椒',
 '来安',
 '天长',
 '琅琊',
 '阜南',
 '颍上',
 '临泉',
 '界首',
 '太和',
 '颍州',
 '颍泉',
 '长丰',
 '肥东',
 '肥西',
 '庐阳',
 '居巢',
 '瑶海',
 '包河',
 '濉溪',
 '烈山',
 '杜集',
 '凤台',
 '潘集',
 '大通',
 '谢家集

In [116]:
import datetime
datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

'2022-08-12 09:11:49'